In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
train_csv = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv')
print(train_csv.info())
print("\n" + "-"*50 + "\n")
train_csv.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4405 entries, 0 to 4404
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   SeriesInstanceUID                           4405 non-null   object
 1   PatientAge                                  4405 non-null   int64 
 2   PatientSex                                  4405 non-null   object
 3   Modality                                    4405 non-null   object
 4   Left Infraclinoid Internal Carotid Artery   4405 non-null   int64 
 5   Right Infraclinoid Internal Carotid Artery  4405 non-null   int64 
 6   Left Supraclinoid Internal Carotid Artery   4405 non-null   int64 
 7   Right Supraclinoid Internal Carotid Artery  4405 non-null   int64 
 8   Left Middle Cerebral Artery                 4405 non-null   int64 
 9   Right Middle Cerebral Artery                4405 non-null   int64 
 10  Anterior Communicating A

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,64,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [3]:
train_csv['Modality'].value_counts()

Modality
CTA           1857
MRA           1256
MRI T2         986
MRI T1post     306
Name: count, dtype: int64

In [4]:
train_localizers_csv = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv')
print(train_localizers_csv.info())
print("\n" + "-"*50 + "\n")
train_localizers_csv.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2286 entries, 0 to 2285
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SeriesInstanceUID  2286 non-null   object
 1   SOPInstanceUID     2286 non-null   object
 2   coordinates        2286 non-null   object
 3   location           2286 non-null   object
dtypes: object(4)
memory usage: 71.6+ KB
None

--------------------------------------------------



,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,"{'x': 189.23979878597123, 'y': 209.19184886465...",Right Middle Cerebral Artery


## 1. 標籤讀取

In [5]:
import pandas as pd
from ast import literal_eval

# 讀取多標籤分類的主標籤
train_csv = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv')
# 我們只保留 SeriesInstanceUID + 14 個 target 欄位
targets = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]
df_labels = train_csv[['SeriesInstanceUID'] + targets].set_index('SeriesInstanceUID')

# 讀取局部定位 (可選，用於定位或 attention map)
loc_csv = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv')
# 把 coordinates (字串) 轉成 tuple
loc_csv['coordinates'] = loc_csv['coordinates'].apply(literal_eval)
# group by series，方便查詢同一序列下的所有 lesion
loc_dict = loc_csv.groupby('SeriesInstanceUID').apply(
    lambda d: list(zip(d['SOPInstanceUID'], d['coordinates'], d['location']))
).to_dict()


/tmp/ipykernel_36/3896420245.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  loc_dict = loc_csv.groupby('SeriesInstanceUID').apply(


In [6]:
df_labels.loc[loc_csv['SeriesInstanceUID'][0]].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

## 2. DICOM 序列前處理

In [7]:
import os
import glob
import numpy as np
import pydicom
import scipy.ndimage

def load_dicom_series(series_uid: str,
                      series_root: str,
                      target_spacing=(1.0, 1.0, 1.0),
                      window_center=40,
                      window_width=300,
                      normalize=True):
    """
    - series_uid: SeriesInstanceUID
    - series_root: e.g. '/kaggle/input/.../series'
    - target_spacing: (z, y, x) in mm
    - window_center/width: CT 窗位
    """
    # 1. 找到該序列所有 DICOM 路徑
    folder = os.path.join(series_root, series_uid)
    dicom_paths = sorted(glob.glob(os.path.join(folder, '*.dcm')))
    # 2. 讀取所有 slice，並排序
    slices = [pydicom.dcmread(p) for p in dicom_paths]
    # 依照 InstanceNumber 或 ImagePositionPatient[2] 排序
    slices.sort(key=lambda s: float(s.InstanceNumber))
    # 3. 堆成 volume
    pixel_arrays = np.stack([s.pixel_array.astype(np.float32) for s in slices], axis=0)
    # 4. 窗寬窗位 (CT)
    wc, ww = window_center, window_width
    lower = wc - ww/2
    upper = wc + ww/2
    pixel_arrays = np.clip(pixel_arrays, lower, upper)
    # 5. 正規化到 [0,1]
    if normalize:
        pixel_arrays = (pixel_arrays - lower) / (upper - lower)
    # 6. 取得目前 spacing
    zs = float(slices[0].SliceThickness)
    ys, xs = [float(x) for x in slices[0].PixelSpacing]
    current_spacing = (zs, ys, xs)
    # 7. 重採樣到 target_spacing
    resize_factor = np.array(current_spacing) / np.array(target_spacing)
    new_shape = np.round(pixel_arrays.shape * resize_factor).astype(int)
    real_resize = new_shape / pixel_arrays.shape
    volume = scipy.ndimage.zoom(pixel_arrays, real_resize, order=1)
    return volume  # shape = (new_slices, new_h, new_w)


In [8]:
import os
import glob
import numpy as np
import pandas as pd
import pydicom
import scipy.ndimage
from ast import literal_eval
import torch
from torch.utils.data import Dataset, DataLoader

# ----------------------------------------
# 1. 讀取標籤
# ----------------------------------------
train_csv = pd.read_csv(
    '/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv'
)
targets = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]
df_labels = train_csv.set_index('SeriesInstanceUID')[targets]

# （可選）讀 train_localizers 做 attention 之用
loc_csv = pd.read_csv(
    '/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv'
)
loc_csv['coordinates'] = loc_csv['coordinates'].apply(literal_eval)
loc_dict = loc_csv.groupby('SeriesInstanceUID').apply(
    lambda d: list(zip(d['SOPInstanceUID'], d['coordinates'], d['location']))
).to_dict()


# ----------------------------------------
# 2. DICOM 序列前處理函式
# ----------------------------------------
def load_dicom_series(
    series_uid: str,
    series_root: str = '/kaggle/input/rsna-intracranial-aneurysm-detection/series',
    target_spacing: tuple = (1.0, 1.0, 1.0),
    window_center: float = 40.0,
    window_width: float = 300.0,
    normalize: bool = True,
):
    """
    1) 讀取並按 InstanceNumber 排序所有 slice
    2) 窗寬窗位 (windowing)：CT 用 window_center, window_width
       └ window_center: 亮度中心 (HU)
       └ window_width: 範圍 (HU)
    3) clip & normalize 到 [0,1]
    4) 重採樣到 target_spacing (z,y,x) mm
    """
    folder = os.path.join(series_root, series_uid)
    paths = sorted(glob.glob(os.path.join(folder, '*.dcm')))
    slices = [pydicom.dcmread(p) for p in paths]
    slices.sort(key=lambda s: float(s.InstanceNumber))

    volume = np.stack([s.pixel_array.astype(np.float32) for s in slices], axis=0)

    # ———— 窗寬窗位說明 ————
    # window_center, window_width 用來限制 HU 範圍：
    #   low = center - width/2, high = center + width/2
    # 這樣 vessels 會更清晰。不同 modality(CTA/MRA/T1/T2) 可以各自調。
    low = window_center - window_width / 2
    high = window_center + window_width / 2
    volume = np.clip(volume, low, high)

    if normalize:
        volume = (volume - low) / (high - low)

    # ———— Resample 說明 ————
    # 因為不同序列的 slice thickness / pixel spacing 可能不一，
    # 我們統一 resample 到 (z,y,x) = target_spacing(mm)
    zs, ys, xs = float(slices[0].SliceThickness), *map(float, slices[0].PixelSpacing)
    current_spacing = (zs, ys, xs)
    resize_factor = np.array(current_spacing) / np.array(target_spacing)
    new_shape = np.round(volume.shape * resize_factor).astype(int)
    real_factor = new_shape / volume.shape
    volume = scipy.ndimage.zoom(volume, real_factor, order=1)

    return volume  # shape = (D, H, W), dtype=float32


# ----------------------------------------
# 3. 自訂 Dataset
# ----------------------------------------
class AneurysmDataset(Dataset):
    def __init__(
        self,
        label_df: pd.DataFrame,
        series_root: str,
        uids: list,
        target_spacing: tuple,
        window_center: float,
        window_width: float,
        modality: str = None,
        transform=None
    ):
        self.label_df = label_df
        self.series_root = series_root
        self.uids = uids
        self.target_spacing = target_spacing
        self.window_center = window_center
        self.window_width = window_width
        self.transform = transform

    def __len__(self):
        return len(self.uids)

    def __getitem__(self, idx):
        uid = self.uids[idx]
        volume = load_dicom_series(
            series_uid=uid,
            series_root=self.series_root,
            target_spacing=self.target_spacing,
            window_center=self.window_center,
            window_width=self.window_width,
        )

        # ———— 這裡示範取中間 3 片當作 3-channel input，如果你要 MCx (2D+1D) 或 3D請調整 ——  
        D, H, W = volume.shape
        mid = D // 2
        img = volume[mid-1:mid+2]  # shape (3, H, W)
        img = torch.from_numpy(img).float()

        label = self.label_df.loc[uid].values.astype(np.float32)
        label = torch.from_numpy(label)

        if self.transform:
            img = self.transform(img)

        return img, label


# ----------------------------------------
# 4. 建立 DataLoader 範例
# ----------------------------------------
all_uids = df_labels.index.tolist()
ds = AneurysmDataset(
    label_df=df_labels,
    series_root='/kaggle/input/rsna-intracranial-aneurysm-detection/series',
    uids=all_uids,
    target_spacing=(1.0, 1.0, 1.0),    # ← 需要你決定
    window_center=40.0,               # ← 需要你決定
    window_width=300.0,               # ← 需要你決定
)
loader = DataLoader(ds, batch_size=8, shuffle=True, num_workers=4)


/tmp/ipykernel_36/3863972519.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  loc_dict = loc_csv.groupby('SeriesInstanceUID').apply(


In [9]:
# 如果你還沒裝 pydicom，可以先執行：
# !pip install pydicom

import pydicom

# DICOM 檔的完整路徑
dcm_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450.dcm'

# 讀檔
ds = pydicom.dcmread(dcm_path)

# 取出像素資料（已經是 numpy array）
img = ds.pixel_array

# 印出 shape
print("Image shape:", img.shape)

# 可以同時看一下 Rows, Columns
print("Rows × Columns:", ds.Rows, "×", ds.Columns)


Image shape: (512, 512)
Rows × Columns: 512 × 512


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import pydicom
import scipy.ndimage
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from ast import literal_eval
import torchvision.models.video as video_models
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# ----------------------------------------
# 1. 標籤讀取
# ----------------------------------------
train_csv = pd.read_csv(
    '/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv'
)
targets = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]
df_labels = train_csv.set_index('SeriesInstanceUID')[targets]

# ----------------------------------------
# 2. DICOM 前處理函式
# ----------------------------------------
def load_dicom_series(
    series_uid: str,
    series_root: str = '/kaggle/input/rsna-intracranial-aneurysm-detection/series',
    target_spacing: tuple = (1.0, 1.0, 1.0),
    apply_windowing: bool = False,
    window_center: float = 40.0,
    window_width: float = 300.0,
    normalize: bool = True
) -> np.ndarray:
    folder = os.path.join(series_root, series_uid)
    dicom_files = sorted(glob.glob(os.path.join(folder, '*.dcm')))
    if not dicom_files:
        raise FileNotFoundError(f"No DICOM files for series {series_uid}")
    slices = [pydicom.dcmread(p, stop_before_pixels=False) for p in dicom_files]
    slices.sort(key=lambda s: float(getattr(s, 'InstanceNumber', 0)))
    volume = np.stack([s.pixel_array.astype(np.float32) for s in slices], axis=0)

    if apply_windowing:
        low = window_center - window_width / 2
        high = window_center + window_width / 2
        volume = np.clip(volume, low, high)
        if normalize:
            volume = (volume - low) / (high - low)

    zs = float(getattr(slices[0], 'SliceThickness', 1.0))
    ys, xs = map(float, getattr(slices[0], 'PixelSpacing', [1.0, 1.0]))
    current_spacing = (zs, ys, xs)
    factor = np.array(current_spacing) / np.array(target_spacing)
    new_shape = np.round(volume.shape * factor).astype(int)
    zoom_factor = new_shape / volume.shape
    volume = scipy.ndimage.zoom(volume, zoom_factor, order=0)
    return volume

# ----------------------------------------
# 3. 自訂 Dataset
# ----------------------------------------
class AneurysmDataset(Dataset):
    def __init__(
        self,
        label_df,
        series_root,
        uids,
        num_slices: int = 32,
        slice_size: tuple = (256, 256),
        target_spacing: tuple = (1.0, 1.0, 1.0),
        apply_windowing: bool = False,
        window_center: float = 40.0,
        window_width: float = 300.0,
        transform=None
    ):
        self.labels = label_df
        self.series_root = series_root
        self.uids = uids
        self.num_slices = num_slices
        self.slice_height, self.slice_width = slice_size
        self.target_spacing = target_spacing
        self.apply_windowing = apply_windowing
        self.window_center = window_center
        self.window_width = window_width
        self.transform = transform

    def __len__(self):
        return len(self.uids)

    def __getitem__(self, idx):
        uid = self.uids[idx]
        try:
            volume = load_dicom_series(
                series_uid=uid,
                series_root=self.series_root,
                target_spacing=self.target_spacing,
                apply_windowing=self.apply_windowing,
                window_center=self.window_center,
                window_width=self.window_width
            )
        except:
            volume = np.zeros((self.num_slices, self.slice_height, self.slice_width), dtype=np.float32)

        D, H, W = volume.shape
        T = self.num_slices
        if D >= T:
            idxs = np.linspace(0, D - 1, T).astype(int)
            vol = volume[idxs]
        else:
            vol = np.zeros((T, H, W), dtype=np.float32)
            vol[:D] = volume

        if (H, W) != (self.slice_height, self.slice_width):
            zoom = (1, self.slice_height / H, self.slice_width / W)
            vol = scipy.ndimage.zoom(vol, zoom, order=0)

        img = torch.from_numpy(vol).unsqueeze(0)
        label = torch.tensor(self.labels.loc[uid].values, dtype=torch.float32)
        if self.transform:
            img = self.transform(img)
        return img, label

# ----------------------------------------
# 4. 訓練函式與迴圈
# ----------------------------------------
def train_one_epoch(loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(loader, desc='Train'): 
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)


def validate_one_epoch(loader, model, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc='Val'):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
    return val_loss / len(loader.dataset)


def main_train():
    # 分割 train/val
    all_uids = df_labels.index.tolist()
    train_uids, val_uids = train_test_split(all_uids, test_size=0.2, random_state=42)

    train_ds = AneurysmDataset(df_labels, '/kaggle/input/rsna-intracranial-aneurysm-detection/series',
                                train_uids, num_slices=32, slice_size=(256,256))
    val_ds = AneurysmDataset(df_labels, '/kaggle/input/rsna-intracranial-aneurysm-detection/series',
                              val_uids, num_slices=32, slice_size=(256,256))
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=0, pin_memory=False)
    val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=0, pin_memory=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = video_models.mc3_18(pretrained=True)
    model.stem[0] = nn.Conv3d(1,64,(3,7,7),(1,2,2),(1,3,3),bias=False)
    model.fc = nn.Linear(model.fc.in_features, len(targets))
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    epochs = 1

    best_val = float('inf')
    for epoch in range(1, epochs+1):
        train_loss = train_one_epoch(train_loader, model, criterion, optimizer, device)
        val_loss = validate_one_epoch(val_loader, model, criterion, device)
        scheduler.step()
        print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        # 儲存最佳模型
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), 'best_aneurysm.pth')
            print("Saved Best Model")

if __name__ == '__main__':
    main_train()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MC3_18_Weights.KINETICS400_V1`. You can also use `weights=MC3_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mc3_18-a90a0ba3.pth" to /root/.cache/torch/hub/checkpoints/mc3_18-a90a0ba3.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 151MB/s] 
Train:  33%|███▎      | 582/1762 [2:04:48<3:21:24, 10.24s/it]/usr/local/lib/python3.11/dist-packages/pydicom/pixels/utils.py:222: UserWarning: A value of 'None' for (0028,0008) 'Number of Frames' is

- IMG shape: (2, 1, 64, 512, 512)
- Lab shape: (2, 14)
- "Out shape: (2, 14)

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import scipy.ndimage
import pydicom
from torchvision import models

# ----------------------------------------
# 1. 定義與訓練時相同的 Dataset
# ----------------------------------------
class AneurysmDataset(Dataset):
    def __init__(self, series_root, uids, num_slices=32, slice_size=(256,256), target_spacing=(1.0,1.0,1.0)):
        self.series_root = series_root
        self.uids = uids
        self.num_slices = num_slices
        self.slice_height, self.slice_width = slice_size
        self.target_spacing = target_spacing

    def __len__(self):
        return len(self.uids)

    def load_series(self, series_uid):
        folder = os.path.join(self.series_root, series_uid)
        paths = sorted(glob.glob(os.path.join(folder, '*.dcm')))
        slices = [pydicom.dcmread(p, stop_before_pixels=False) for p in paths]
        slices.sort(key=lambda s: float(getattr(s, 'InstanceNumber', 0)))
        volume = np.stack([s.pixel_array.astype(np.float32) for s in slices], axis=0)
        # Resample to target_spacing
        zs = float(getattr(slices[0], 'SliceThickness',1.0))
        ys, xs = map(float, getattr(slices[0], 'PixelSpacing',[1.0,1.0]))
        factor = np.array((zs,ys,xs)) / np.array(self.target_spacing)
        new_shape = np.round(volume.shape * factor).astype(int)
        zoom = new_shape / volume.shape
        volume = scipy.ndimage.zoom(volume, zoom, order=0)
        return volume

    def __getitem__(self, idx):
        uid = self.uids[idx]
        volume = self.load_series(uid)
        D, H, W = volume.shape
        T = self.num_slices
        # sample or pad
        if D >= T:
            indices = np.linspace(0, D-1, T).astype(int)
            vol = volume[indices]
        else:
            vol = np.zeros((T,H,W), dtype=np.float32)
            vol[:D] = volume
        # resize slices
        if (H,W) != (self.slice_height, self.slice_width):
            zoom2 = (1, self.slice_height/H, self.slice_width/W)
            vol = scipy.ndimage.zoom(vol, zoom2, order=0)
        img = torch.from_numpy(vol).unsqueeze(0)
        return uid, img

# ----------------------------------------
# 2. 載入模型並推論
# ----------------------------------------
def inference_and_submit(model_path: str,
                         test_series_root: str,
                         output_csv: str = 'submission.csv',
                         batch_size: int = 1):
    # Targets 順序需保持一致
    targets = [
        'Left Infraclinoid Internal Carotid Artery',
        'Right Infraclinoid Internal Carotid Artery',
        'Left Supraclinoid Internal Carotid Artery',
        'Right Supraclinoid Internal Carotid Artery',
        'Left Middle Cerebral Artery',
        'Right Middle Cerebral Artery',
        'Anterior Communicating Artery',
        'Left Anterior Cerebral Artery',
        'Right Anterior Cerebral Artery',
        'Left Posterior Communicating Artery',
        'Right Posterior Communicating Artery',
        'Basilar Tip',
        'Other Posterior Circulation',
        'Aneurysm Present'
    ]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 建立 MC3_18 架構
    model = models.video.mc3_18(pretrained=False)
    model.stem[0] = nn.Conv3d(1,64,(3,7,7),(1,2,2),(1,3,3),bias=False)
    model.fc = nn.Linear(model.fc.in_features, len(targets))
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device).eval()

    # 測試集
    test_uids = sorted(os.listdir(test_series_root))
    test_ds = AneurysmDataset(test_series_root, test_uids)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    results = []
    with torch.no_grad():
        for uids, imgs in test_loader:
            imgs = imgs.to(device)
            logits = model(imgs)
            probs = torch.sigmoid(logits).cpu().numpy()
            for uid, p in zip(uids, probs):
                results.append([uid] + p.tolist())

    cols = ['SeriesInstanceUID'] + targets
    sub_df = pd.DataFrame(results, columns=cols)
    sub_df.to_csv(output_csv, index=False)
    print(f"Saved {output_csv}")

if __name__ == '__main__':
    inference_and_submit(
        model_path='best_aneurysm.pth',
        test_series_root='/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation/series',
        output_csv='submission.csv',
        batch_size=1
    )
